In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import wfdb
from wfdb import processing

from gc import collect as collect_garbage
from psutil import virtual_memory
from os import scandir

In [17]:
#folder = "D:/nicu-hrv-data/00-deidentified-raw-waveforms/"
folder = "C:/Users/aidan/Box/Deidentified-Raw-Waveforms/"
coldict = {
    "raw_waves_data_1a.csv": ["time", "257"], "raw_waves_data_1b.csv": ["time", "257", "258"], "raw_waves_data_1c.csv": ["time", "257", "258"], "raw_waves_data_1d.csv": ["time", "257", "258", "317"], 
    "raw_waves_data_1e.csv": ["time", "258"],

    "raw_waves_data_2a.csv": ["time", "257", "258"], "raw_waves_data_2b.csv": ["time", "258"], "raw_waves_data_2c.csv": ["time", "257"], "raw_waves_data_2d.csv": ["time", "257", "258"], 
    "raw_waves_data_2e.csv": ["time", "257", "258"],

    "raw_waves_data_3a.csv": ["time", "258"], "raw_waves_data_3b.csv": ["time", "258"], "raw_waves_data_3c.csv": ["time", "258"], "raw_waves_data_3d.csv": ["time", "258"], 
    "raw_waves_data_3e.csv": ["time", "257", "258", "317"],

    "raw_waves_data_4a.csv": ["time", "257", "258"], "raw_waves_data_4b.csv": ["time", "257", "258"], "raw_waves_data_4c.csv": ["time", "257"], "raw_waves_data_4d.csv": ["time", "257", "258"], 
    "raw_waves_data_4e.csv": ["time", "257", "258"],

    "raw_waves_data_5a.csv": ["time", "258"], "raw_waves_data_5b.csv": ["time", "258"], "raw_waves_data_5c.csv": ["time", "258"], "raw_waves_data_5d.csv": ["time", "258", "317"],
    "raw_waves_data_5e.csv": ["time", "258"],

    "raw_waves_data_6a.csv": ["time", "257", "258"], "raw_waves_data_6b.csv": ["time", "258"], "raw_waves_data_6c.csv": ["time", "258"], "raw_waves_data_6d.csv": ["time", "258"], "raw_waves_data_6e.csv": ["time", "258"],
    
    "raw_waves_data_7a.csv": ["time", "257", "258"], "raw_waves_data_7b.csv": ["time", "258"], "raw_waves_data_7c.csv": ["time", "258"], "raw_waves_data_7d.csv": ["time", "257", "258", "317"], 
    "raw_waves_data_7e.csv": ["time", "258"]
}

namedict = {
    "raw_waves_data_1a.csv": "1a", "raw_waves_data_1b.csv": "1b", "raw_waves_data_1c.csv": "1c", "raw_waves_data_1d.csv": "1d", "raw_waves_data_1e.csv": "1e",
    "raw_waves_data_2a.csv": "2a", "raw_waves_data_2b.csv": "2b", "raw_waves_data_2c.csv": "2c", "raw_waves_data_2d.csv": "2d", "raw_waves_data_2e.csv": "2e",
    "raw_waves_data_3a.csv": "3a", "raw_waves_data_3b.csv": "3b", "raw_waves_data_3c.csv": "3c", "raw_waves_data_3d.csv": "3d", "raw_waves_data_3e.csv": "3e",
    "raw_waves_data_4a.csv": "4a", "raw_waves_data_4b.csv": "4b", "raw_waves_data_4c.csv": "4c", "raw_waves_data_4d.csv": "4d", "raw_waves_data_4e.csv": "4e",
    "raw_waves_data_5a.csv": "5a", "raw_waves_data_5b.csv": "5b", "raw_waves_data_5c.csv": "5c", "raw_waves_data_5d.csv": "5d", "raw_waves_data_5e.csv": "5e",
    "raw_waves_data_6a.csv": "6a", "raw_waves_data_6b.csv": "6b", "raw_waves_data_6c.csv": "6c", "raw_waves_data_6d.csv": "6d", "raw_waves_data_6e.csv": "6e",
    "raw_waves_data_7a.csv": "7a", "raw_waves_data_7b.csv": "7b", "raw_waves_data_7c.csv": "7c", "raw_waves_data_7d.csv": "7d", "raw_waves_data_7e.csv": "7e"    
}

In [18]:
collect_garbage()
virtual_memory()

svmem(total=12655771648, available=5600448512, percent=55.7, used=7055323136, free=5600448512)

In [19]:
all_files = [folder + file.name for file in scandir(folder) if ".csv" in file.name]

In [20]:
# Takes a file name, gets the key needed to index the above dictionaries
def get_key(file_name):
    return file_name.split("/")[-1]

# Takes a dataframe of ECG signals, completes the signals, fill in missing data with ffill, 
# and returns a dataframe containing the new signal
def complete_signal(df):
    cols = df.columns
    signal = pd.Series(df[cols[1]])
    i=2
    while True:
        try:
            signal = signal.combine_first(df[cols[i]])
        except IndexError:
            break
    signal = signal.fillna(method="ffill")
    signal = pd.to_numeric(signal)
    return signal

# Takes signal, and modifies it in place to have no spikes or troughs
def remove_extrema(signal):
    delta = 125
    filt = (signal <= -10) | (signal >= 10)
    filt.loc[~filt] = np.nan
    filt.fillna(method="ffill", limit=delta, inplace=True)
    filt.fillna(method="bfill", limit=delta, inplace=True)
    filt.fillna(value=False, inplace=True)
    
    signal.loc[filt] = np.nan
    signal.fillna(method="ffill", inplace=True)

# Takes a signal and returns a list of qrs locations, as found by XQRS
def detect_qrs(signal, freq=250, chunk=10000):
    # Initialize the rpeak list
    rpeaks = []

    # Create a counter for breaking the signal into chunks
    i=0
    N = len(signal)
    chunk = 10000
    num_chunks = N//chunk + 1

    # Find R peaks in all but the last chunk (that just tends to cause a problem)
    while True:
        try:
            if i%1000 == 0:
                # I've found this choice of progress marker works for this chunk
                # size and signal length. If those values change, then this 
                # condition will need to be modified too
                print( str(round(i/num_chunks,4)*100) + "% percent done" )

            lo = i*chunk
            hi = min( (i+1)*chunk, N)
            xqrs = processing.XQRS(sig=signal[lo:hi], fs=freq)
            xqrs.detect(verbose=False)

            # xqrs recognized the chunk as starting from 0, so we have to shift 
            # the R peaks according to the left endpoint of the chunk
            rpeaks += list( lo + xqrs.qrs_inds )

            i+=1
        except IndexError:
            # This is the main way in which we'd expect to break this loop
            break
        except ValueError:
            # More often than not, we get this case because the last chunk isn't 
            # long enough, hence the next block
            break
    print("R peaks outside of the last chunk located")

    # Delineate an ending chunk of like 20000 indices that gets the end of the
    # signal, find R peaks
    hi = len(signal)
    lo = hi - 20000
    xqrs = processing.XQRS(sig=signal[lo:hi], fs=freq)
    xqrs.detect(verbose=False)

    rpeaks += [ peak for peak in xqrs.qrs_inds if peak > max(rpeaks)]

    return rpeaks

# Removes RR intervals that are too small and returns a new dataframe with 
# RR intervals greater than that threshold
def remove_too_small(df, lo=0.25):
    filt = df["interval"] >= lo

    new_df = pd.DataFrame.copy(df.loc[filt], deep=True)
    new_df.reset_index(inplace=True, drop=True)

    new_df["interval"] = new_df["time"].diff()

    return new_df

# Removes RR intervals that are too big (inplace), just treats them as missing data
def remove_too_big(df, hi=5):
    filt = df["interval"] <= 5
    df = df.loc[filt]
    df.reset_index(inplace=True, drop=True)
    
# Breaks up any multiple intervals
def remove_multiple_intervals(df):
    # I have the code for this at home and can implement it when I get there
    return
    

In [21]:
for i in range(1,8):
    files = [file for file in all_files if "_"+str(i) in file]

    peak_df = None
    for file in files:
        # Preliminaries
        key = get_key(file)
        cols = coldict[key]
        freq = 250
        print("Starting now with file " + key)
        print(str(virtual_memory()[2]) + "% memory usage")

        # Read in the data
        df = pd.read_csv(file, usecols=cols)
        print("Raw data loaded in")
        print(str(virtual_memory()[2]) + "% memory usage")
        
        # Get a workable signal
        signal = complete_signal(df)
        remove_extrema(signal)
        print("Signal extracted from raw data")
        print(str(virtual_memory()[2]) + "% memory usage")

        # Detect QRS complexes
        rpeaks = detect_qrs(signal, freq=250, chunk=10000)
        print("QRS complexes located")
        print(str(virtual_memory()[2]) + "% memory usage")

        if peak_df==None:
            peak_df = df.loc[rpeaks, "time"]
        else:
            peak_df = peak_df.append(df.loc[rpeaks, "time"])
        del df
        collect_garbage()

    peak_df.reset_index(inplace=True)

    # Write the peaks to an output file
    peak_df.to_csv("D:/nicu-hrv-data/01-rpeaks/rpeaks_" + str(i) + ".csv", index=False)

    # Clean up
    del peak_df
    collect_garbage()
    print("Done with infant " + str(i))
    print(str(virtual_memory()[2]) + "% memory usage")
    


Starting now with file raw_waves_data_1a.csv
55.8% memory usage
Raw data loaded in
58.1% memory usage
Signal extracted from raw data
60.9% memory usage
0.0% percent done


In [ ]:
# Flag any beats that are < 0.25 seconds, remove them, and recalculate the RR intervals


In [ ]:
# ALRIGHT, PART ONE IS DONE, NOW I NEED TO FIX THE RR INTERVALS TO REMOVE DOUBLE INTERVALS, WILL ALSO TAKE A WHILE
# Load in the rpeaks for each infant
for i in range(2,8):
    print("Starting now with infant " + str(i))
    # Get the names of the rpeak files
    rpeak_files = sorted(["07-pipeline-outputs/01-rpeaks/rpeaks_"+str(i)+part+".csv" for part in ["a", "b", "c", "d", "e"]])
    
    # Concatenate all of these dataframes
    df = pd.read_csv( rpeak_files.pop(0) )
    for file in rpeak_files:
        df = df.append( pd.read_csv( file ) )
        collect_garbage()
    print("Infant " + str(i) + " R-peaks all loaded")
    
    # Clean up df, calculate the RR intervals, and write the results just in case
    df.drop("Unnamed: 0", inplace=True, axis=1)
    df.reset_index(inplace=True, drop=True)
    df["interval"] = df["time"].diff()
    df.to_csv("07-pipeline-outputs/02-dirty-rr-intervals/dirty-rr-intervals_"+str(i)+".csv", index=False)
    print("Unprocessed RR Intervals Written")

    # Recalculate RR intervals while ignoring beats whose RR intervals are > 0.25 seconds
    filt = df["interval"] >= 0.25

    new_df = pd.DataFrame.copy(df.loc[filt], deep=True)
    new_df.reset_index(inplace=True, drop=True)

    new_df["interval"] = new_df["time"].diff()

    del df
    collect_garbage()
    df = pd.DataFrame.copy(new_df, deep=True)
    del new_df
    collect_garbage()

    # Remove intervals of length greater than 5 seconds (this is arbitrary)
    # The resulting gaps will just be treated as missing data
    filt = df["interval"] <= 5
    df = df.loc[filt]
    df.reset_index(inplace=True, drop=True)
    print("Intervals outside of [0.25, 10] dealt with")

    # Get rid of any remaining multiple intervals
    df.set_index(df["time"], inplace=True)
    df.drop("time", inplace=True, axis=1)

    df_buffer = pd.DataFrame.copy(df, deep=True)

    # THIS IS THE BULKY PART OF THIS PROCESS, AND IT DOES TAKE A LONG TIME (~1H PER INFANT).
    # INVESTIGATION INTO OTHER METHODS WOULD BE WORTHWHILE
    # Form a generator to iterate over the rows
    rows = df_buffer.iterrows()
    progress_chunk = len(df["interval"]) // 20

    # Get the initial values
    prev_idx, prev_row = next(rows)
    prev_ivl = prev_row["interval"]

    # A counter to see how many beats were imputed
    imputed = 0
    max_imputed = 0

    # A counter for progress
    counter = 0

    for curr_idx, curr_row in rows:
        if counter % progress_chunk == 0:
            print( str(round( counter/progress_chunk, 4)*100) + "% Complete" )
        counter += 1


        curr_ivl = curr_row["interval"]

        pieces = round(curr_ivl/prev_ivl)

        if pieces >= 2: # then it is likely that the current interval is a multiple interval
            fill_value = curr_ivl / pieces

            while fill_value < 0.25: # We have too many pieces and the beats are unrealistically small (i.e. < 0.25 seconds)
                pieces -= 1
                fill_value = curr_ivl/pieces
                if pieces == 1:
                    break
                
            if pieces == 1: # Then there's not point in carrying on with this iteration, update previous values and continue
                prev_idx = curr_idx
                prev_ivl = curr_ivl
                continue

            imputed += pieces
            max_imputed = max(max_imputed, pieces)

            # Otherwise, we impute the RR intervals, modifying df and NOT df_buffer
            endpoints = [prev_idx + i*fill_value for i in range(1, pieces)] + [curr_idx]
            for t in endpoints:
                df.loc[t,"interval"] = fill_value

            # Now, we update the previous values ahead of the next iteration
            prev_idx = curr_idx
            prev_ivl = fill_value
            continue

        # If we didn't enter the pieces >= 2 case, then we need to update the previous values in a different way
        prev_idx = curr_idx
        prev_ivl = curr_ivl

    print("Multiple intervals all broken up")
    df.sort_index(inplace=True)
    
    df.to_csv("07-pipeline-outputs/03-clean-rr-intervals/rr_intervals_imputed_"+str(i)+".csv")
    del df
    del df_buffer
    del rows
    collect_garbage()
    print("Cleaned RR intervals written to file, "+str(virtual_memory()[2])+"% memory usage\n")
